In [1]:
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import re
import string
import operator
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
import numpy as np
import os

In [2]:
def process_text(data):
    #data=str(raw_data).encode('ascii', 'ignore')
    letters_only = re.sub("[^a-zA-Z]", " ", data)
    #print letters_only
    lower_case = letters_only.lower()
    words = lower_case.split()
    cachedStopWords=stopwords.words("english")
    stop_word = [w for w in words if not w in cachedStopWords]
    #print stop_word
    exclude = set(string.punctuation)
    text = [ch for ch in stop_word if ch not in exclude]
    #print text
    #stemmer=PorterStemmer()
    #stemmed_text=(stemmer.stem(text))
    #a = WordNetLemmatizer()
    #lemmatized_text=a.lemmatize(stemmed_text)
    processed_text = (" ".join(text))
    processed_text = ' '.join(word for word in processed_text.split() if 15>len(word)>3)
    return processed_text

In [3]:
#TRAINING
#READ FILES AND CREATE MEGA DOCUMENT FOR EACH CATEGORY
train_path='/Users/VA/Documents/MS/Acads/Fall15/ML/20_newsgroups_final/training'
mega_doc={}
overall_text=''
overall_text_list = []
for category in os.listdir(train_path):
    category_text=''
    if (category!= '.DS_Store'):
        print 'Creating mega document for', category, '\n'
        for filename in os.listdir(train_path + '/' + category):
            f=open(train_path + '/' + category + '/' + filename, 'r');
            category_text = category_text + f.read()
            f.close()
        overall_text = overall_text+category_text
        overall_text=process_text(overall_text)
        overall_text_list.append(process_text(overall_text))
        mega_doc[category] = process_text(category_text)
print 'Mega Documents created.'

Creating mega document for alt.atheism 

Creating mega document for comp.graphics 

Creating mega document for comp.os.ms-windows.misc 

Creating mega document for comp.sys.ibm.pc.hardware 

Creating mega document for comp.sys.mac.hardware 

Creating mega document for comp.windows.x 

Creating mega document for misc.forsale 

Creating mega document for rec.autos 

Creating mega document for rec.motorcycles 

Creating mega document for rec.sport.baseball 

Creating mega document for rec.sport.hockey 

Creating mega document for sci.crypt 

Creating mega document for sci.electronics 

Creating mega document for sci.med 

Creating mega document for sci.space 

Creating mega document for soc.religion.christian 

Creating mega document for talk.politics.guns 

Creating mega document for talk.politics.mideast 

Creating mega document for talk.politics.misc 

Creating mega document for talk.religion.misc 

Mega Documents created.


In [4]:
#TRAINING
#CALCULATE PRIOR PROBABILITIES
train_path='/Users/VA/Documents/MS/Acads/Fall15/ML/20_newsgroups_final/training'
#CALCULATE OVERALL COUNT
overall_count=0
for category in os.listdir(train_path):
    count=0;
    if (category!= '.DS_Store'):
        for filename in os.listdir(train_path + '/' + category):
            count=count+1
        overall_count=overall_count+count
print 'The overall count is %d'%overall_count
#CALCULATE DOC COUNT AND PRIOR PROBABILITIES
prior_probabilities={}
for category in os.listdir(train_path):
    count=0;
    prior_prob=0
    if (category!= '.DS_Store'):
        print '\nCategory:', category,';'
        for filename in os.listdir(train_path + '/' + category):
            count=count+1
        prior_prob = float(count) / (overall_count)
        print 'Prior Probability:', prior_prob
    prior_probabilities[category]=prior_prob

The overall count is 10000

Category: alt.atheism ;
Prior Probability: 0.05

Category: comp.graphics ;
Prior Probability: 0.05

Category: comp.os.ms-windows.misc ;
Prior Probability: 0.05

Category: comp.sys.ibm.pc.hardware ;
Prior Probability: 0.05

Category: comp.sys.mac.hardware ;
Prior Probability: 0.05

Category: comp.windows.x ;
Prior Probability: 0.05

Category: misc.forsale ;
Prior Probability: 0.05

Category: rec.autos ;
Prior Probability: 0.05

Category: rec.motorcycles ;
Prior Probability: 0.05

Category: rec.sport.baseball ;
Prior Probability: 0.05

Category: rec.sport.hockey ;
Prior Probability: 0.05

Category: sci.crypt ;
Prior Probability: 0.05

Category: sci.electronics ;
Prior Probability: 0.05

Category: sci.med ;
Prior Probability: 0.05

Category: sci.space ;
Prior Probability: 0.05

Category: soc.religion.christian ;
Prior Probability: 0.05

Category: talk.politics.guns ;
Prior Probability: 0.05

Category: talk.politics.mideast ;
Prior Probability: 0.05

Category: t

In [5]:
#TRAINING
#VECTORIZE THE TEXT AND EXTRACT VOCABULARY
data_vectorizer = CountVectorizer(input=overall_text_list, decode_error= 'ignore', analyzer = "word", tokenizer = None, preprocessor = None, stop_words = 'english', max_features = 100000)
train_data_features = data_vectorizer.fit_transform(overall_text_list)
train_data_features_array = train_data_features.toarray()
#print train_data_features
#for i in range(len(train_data_features_array[0])):
#    for j in range(len(train_data_features_array[i])):
#        print train_data_features_array[i][j]
#print train_data_features_array
vocab = data_vectorizer.get_feature_names()
print 'Vocabulary Generated.'
#dist=train_data_features_array.sum(axis=0)
#feature_names = np.asarray(vocab)
#for name in feature_names:
#    print name
#print dist
#for tag, count in zip(vocab, dist):
#    print tag
#    print count

Vocabulary Generated.


In [9]:
#TRAINING
#CALCULATE LIKELIHOOD AND CORRESPONDING DICTIONARY
train_path='/Users/VA/Documents/MS/Acads/Fall15/ML/20_newsgroups_final/training'
likelihood_dict_outer={}
for category in os.listdir(train_path):
    print category
    likelihood_dict_inner={}
    word_count=0
    for word in vocab:
        if (word in mega_doc[category].split()):
            #mega_doc[category].split()
            word_count = mega_doc[category].count(word)
        else:
            word_count=0
        likelihood = float(word_count+1)/((len(mega_doc[category].split()))+len(vocab))
        likelihood_dict_inner[word]=likelihood
    likelihood_dict_outer[category]=likelihood_dict_inner

alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc


In [ ]:
#TESTING
#CALCULATE POSTERIOR PROBABILITY
test_path='/Users/VA/Documents/MS/Acads/Fall15/ML/20_newsgroups_final/testing'
predict_dict={}
for test_filename in os.listdir(test_path):
    if test_filename != ".DS_Store":
        f=open(test_path + '/' + test_filename,'r')
        d = process_text(f.read()).split()
        posterior_probabilities={}
        overall_categories=['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
        for category in overall_categories:
            posterior_likelihood=1
            for word in d:
                if word in vocab:
                    posterior_likelihood=posterior_likelihood*likelihood_dict_outer[category][word]
            posterior_probabilities[category]=((posterior_likelihood)*(prior_probabilities[category]))
        predict_dict[test_filename]=max(posterior_probabilities.iteritems(), key=operator.itemgetter(1))[0]
        f.close()
#print 'Predicted Classes:', predict_dict

In [43]:
#EVALUATION
test_path='/Users/VA/Documents/MS/Acads/Fall15/ML/20_newsgroups_final/testing/'
test_label_path='/Users/VA/Documents/MS/Acads/Fall15/ML/20_newsgroups_final/testing_labels/'
overall_categories=['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
filename_labels={}
for category in overall_categories:
    for filename in os.listdir(test_path):
        if (os.path.isfile(test_label_path+str(category)+ '/' + str(filename)))==True:
            filename_labels[filename]=category
print 'Original Classes:', filename_labels
count=0;

for filename in os.listdir(test_path):
    if (filename != '.DS_Store'):
        if filename_labels[filename]==predict_dict[filename]:
            count=count+1
print '\nNumber of correctly predicted documents=%d'%count
print 'Total number of documents=%d'%len(predict_dict)
accuracy = float(count*100)/len(predict_dict)
print 'Mean Detection Accuracy=%f'%accuracy,'%'        


Original Classes: {'8514': 'comp.os.ms-windows.misc', '9136': 'comp.os.ms-windows.misc'}

Number of correctly predicted documents=5297
Total number of documents=9118
Mean Detection Accuracy=58.093880 %
